# KNN

k개의 주변 이웃 중 가장 많은 값으로 분류

In [1]:
import pandas as pd
import seaborn as sns

In [2]:
df=sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [3]:
df.info()   ##age,embarked,embarked_town,deck 변수에 결측치 존재 -> deck은 제거, embarked=embarked_town이므로 하나 제거

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [4]:
df=df.drop(['deck','embark_town'],axis=1) 
df=df.dropna(subset=['age'],axis=0)      #결측치 행 삭제

In [5]:
df['embarked'].value_counts()

S    554
C    130
Q     28
Name: embarked, dtype: int64

In [6]:
df['embarked'].fillna('s',inplace=True)   #결측치 최빈값으로 처리

In [7]:
ndf=df[['survived','pclass','sex','age','sibsp','parch','embarked']]   #변수 선택

In [8]:
#원핫인코딩
onehot_sex=pd.get_dummies(ndf['sex'])
ndf=pd.concat([ndf,onehot_sex],axis=1)

onthot_embarked=pd.get_dummies(ndf['embarked'])
ndf=pd.concat([ndf,onthot_embarked],axis=1)

ndf.drop(['sex','embarked'],axis=1,inplace=True)
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,C,Q,S,s
0,0,3,22.0,1,0,0,1,0,0,1,0
1,1,1,38.0,1,0,1,0,1,0,0,0
2,1,3,26.0,0,0,1,0,0,0,1,0
3,1,1,35.0,1,0,1,0,0,0,1,0
4,0,3,35.0,0,0,0,1,0,0,1,0


In [9]:
ndf.drop(['s'],axis=1,inplace=True)

In [10]:
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,C,Q,S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


In [24]:
X=ndf[['pclass','age','sibsp','parch','female','male','C','Q','S']]
y=ndf['survived']

from sklearn import preprocessing
X=preprocessing.StandardScaler().fit(X).transform(X)

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=111)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(571, 9)
(571,)
(143, 9)
(143,)


In [25]:
from sklearn.neighbors import KNeighborsClassifier

knn=KNeighborsClassifier(n_neighbors=5)  #이웃의 수는 5
knn.fit(X_train,y_train)
y_hat=knn.predict(X_test)

print(y_hat[0:10])
print(y_test.values[0:10])

[0 0 0 1 0 1 1 0 0 1]
[0 0 0 1 0 1 0 0 1 1]


예측 평가지표
1. 정확도(preicision): true로 예측한 애들 중 true 인 비율
2. 재현율(recall): 실제값이 true 인 애들 중 true로 예측 한 비율
3. F1 지표: 2 x (precision x recall) / (precision + recall)

In [29]:
#confuion matrix
from sklearn import metrics
knn_matrix=metrics.confusion_matrix(y_test,y_hat)
print(knn_matrix)

[[72 16]
 [14 41]]


In [30]:
#평가지표
knn_report=metrics.classification_report(y_test,y_hat)
print(knn_report)

              precision    recall  f1-score   support

           0       0.84      0.82      0.83        88
           1       0.72      0.75      0.73        55

    accuracy                           0.79       143
   macro avg       0.78      0.78      0.78       143
weighted avg       0.79      0.79      0.79       143



# SVM

같은 분류 값을 갖는 데이터끼리 같은 벡터공간에 위치하도록 경계를 찾는 방식

In [34]:
#모델적용
from sklearn import svm

svm_model=svm.SVC(kernel='rbf')   #kernel: 데이터를 벡터공간으로 매핑하는 함수
svm_model.fit(X_train,y_train)
y_hat=svm_model.predict(X_test)

print(y_hat[0:10])
print(y_test.values[0:10])

[0 0 0 1 0 1 0 0 0 1]
[0 0 0 1 0 1 0 0 1 1]


In [35]:
#평가
svm_matrix=metrics.confusion_matrix(y_test,y_hat)
print(svm_matrix)
print('\n')

svm_report=metrics.classification_report(y_test,y_hat)
print(svm_report)

[[76 12]
 [16 39]]


              precision    recall  f1-score   support

           0       0.83      0.86      0.84        88
           1       0.76      0.71      0.74        55

    accuracy                           0.80       143
   macro avg       0.80      0.79      0.79       143
weighted avg       0.80      0.80      0.80       143



# Decision Tree

각 노드마다 목표값을 가장 잘 분류할 수 있는 속성을 찾아 배치하고, 속성마다 새로운 가지를 만드는 방식

In [39]:
from sklearn import tree

tree_model=tree.DecisionTreeClassifier(criterion='entropy',max_depth=5)   #평가기준으로 entropy값을 설정하고 트리레벨은 최대 5
#지나치게 깊은 트리는 과적합 문제 발생 -> 적당한 트리레벨 찾기 필요

tree_model.fit(X_train,y_train)
y_hat=tree_model.predict(X_test)

print(y_hat[0:10])
print(y_test.values[0:10])

[0 0 0 1 0 1 0 0 0 1]
[0 0 0 1 0 1 0 0 1 1]


In [40]:
tree_matrix=metrics.confusion_matrix(y_test,y_hat)
print(tree_matrix)
print('\n')

tree_report=metrics.classification_report(y_test,y_hat)
print(svm_report)

[[78 10]
 [16 39]]


              precision    recall  f1-score   support

           0       0.83      0.86      0.84        88
           1       0.76      0.71      0.74        55

    accuracy                           0.80       143
   macro avg       0.80      0.79      0.79       143
weighted avg       0.80      0.80      0.80       143

